In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
# hyper parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST dataset(image and labels)

train_dataset = torchvision.datasets.MNIST(root = '../../data',
                                         train = True,
                                         transform=transforms.ToTensor(),
                                         download=True
                                         )

test_dataset = torchvision.datasets.MNIST(root = '../../data',
                                         train = False,
                                         transform=transforms.ToTensor(),
                                         )

# data loader(input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [11]:
'''# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(x)
        return out'''
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [8]:
# loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [14]:
# train the model

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # reshape iamges to (batch_size, input_size)
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward and optpimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 ==0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.3044
Epoch [1/5], Step [200/600], Loss: 2.2968
Epoch [1/5], Step [300/600], Loss: 2.2914
Epoch [1/5], Step [400/600], Loss: 2.2965
Epoch [1/5], Step [500/600], Loss: 2.3002
Epoch [1/5], Step [600/600], Loss: 2.2949
Epoch [2/5], Step [100/600], Loss: 2.2977
Epoch [2/5], Step [200/600], Loss: 2.2965
Epoch [2/5], Step [300/600], Loss: 2.2868
Epoch [2/5], Step [400/600], Loss: 2.3010
Epoch [2/5], Step [500/600], Loss: 2.2968
Epoch [2/5], Step [600/600], Loss: 2.2809
Epoch [3/5], Step [100/600], Loss: 2.2896
Epoch [3/5], Step [200/600], Loss: 2.2995
Epoch [3/5], Step [300/600], Loss: 2.2975
Epoch [3/5], Step [400/600], Loss: 2.3020
Epoch [3/5], Step [500/600], Loss: 2.3046
Epoch [3/5], Step [600/600], Loss: 2.3027
Epoch [4/5], Step [100/600], Loss: 2.3004
Epoch [4/5], Step [200/600], Loss: 2.3008
Epoch [4/5], Step [300/600], Loss: 2.2928
Epoch [4/5], Step [400/600], Loss: 2.2980
Epoch [4/5], Step [500/600], Loss: 2.2943
Epoch [4/5], Step [600/600], Loss:

In [15]:
# test the model
# in test phase, don't compute gradients for memory efficiency

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
    



Accuracy of the model on the 10000 test images: 9 %


In [16]:
torch.save(model.state_dict(), 'Neural Network.ckpt')